In [1]:
%pwd

'i:\\19. YT_mac_data_insights\\7.My_microserv_website\\AI_ML_Portfolio\\research'

In [3]:
# The directory is in research folder, Let's move back to the root folder

import os

os.chdir("../") # ../ moves back one folder

In [4]:
%pwd

'i:\\19. YT_mac_data_insights\\7.My_microserv_website\\AI_ML_Portfolio'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# Extracting the documents from the PDF file

def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [8]:
extracted_data=load_pdf_file(data='downloads/')

In [11]:
# extracted_data

In [12]:
# split the data into text chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [13]:
text_chunks=text_split(extracted_data)

In [26]:
len(text_chunks)

671

In [16]:
# Download huggingface embeddings model

from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [17]:
embeddings=download_hugging_face_embeddings()

C:\Users\mahen\AppData\Local\Temp\ipykernel_8792\2323421661.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
i:\19. YT_mac_data_insights\7.My_microserv_website\AI_ML_Portfolio\aienv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
query_result=embeddings.embed_query("welcome")
print(len(query_result))

384


In [20]:
# query_result

#### Storing vector data in Pinecone vector database



In [33]:
from dotenv import load_dotenv
load_dotenv()

True

In [34]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [35]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [27]:
# Loading data into pinecone vector db

from langchain_pinecone import PineconeVectorStore

index_name = "annualreportagent"
docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [28]:
# Loading Existing index

docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [29]:
docsearch

In [30]:
retriever=docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

In [32]:
retriever.invoke("what is the annual revenue?")

[Document(id='1a51445a-b85c-4548-a7af-6cddd03e06a5', metadata={'page': 37.0, 'source': 'downloads\\2023_Annual_Report.pdf'}, page_content='product or service. Refer to Note 1 – Accounting Policies of the Notes to Financial Statements for further discussion.  \nThe following table outlines the expected future recognition of unearned revenue as of June  30, 2023:  \n  \n(In millions)     \n    \nThree Months Ending   \n    \nSeptember 30, 2023 $ 19,673  \nDecember 31, 2023  15,600  \nMarch 31, 2024  10,801  \nJune 30, 2024  4,827  \nThereafter  2,912    \nTotal $   53,813'),
 Document(id='a677252f-c3dc-4dc5-bfcd-633c3d3e8e10', metadata={'page': 68.0, 'source': 'downloads\\2023_Annual_Report.pdf'}, page_content='Recognition of unearned revenue  (118,530)   \nBalance, end of period $   53,813      \nRevenue allocated to remaining performance obligations, which includes unearned revenue and amounts that will be \ninvoiced and recognized as revenue in future periods, was $229 billion as of J

In [36]:
from langchain_openai import OpenAI
llm=OpenAI(temperature=0.4,max_tokens=50)

In [38]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt=(
    "You are a financial auditor who reviewed everything about microsoft financials for question-answering tasks."
    "use the following pieces of retreived context and based on this context provide an answer"
    "If you don't know the answer, say that you don't"
    "know. Use three sentences maximum and keep the answer concise"
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [39]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [41]:
response=rag_chain.invoke({"input":"How many employees are there at microsoft?"})
print(response["answer"])



There are approximately 151,000 employees at Microsoft.
